# Load Packages, Libraries and Configs

* load packages

In [ ]:
from RES.hdf5_handler import DataHandler
import RES.visuals as vis
import RES.utility as utils
import pandas as pd
import geopandas as gpd
import warnings

# Suppress specific warnings
warnings.filterwarnings("ignore", category=UserWarning)

In [ ]:
import matplotlib.pyplot as plt

title_fontsize = 12
axis_labelsize = 10
tick_fontsize = 9
caption_fontsize = 9

plt.rcParams.update({
    'font.family': 'serif',
    'font.size': 9,
    'axes.titlesize': title_fontsize,
    'axes.labelsize': axis_labelsize,
    'xtick.labelsize': tick_fontsize,
    'ytick.labelsize': tick_fontsize,
})


* Load Config

In [ ]:
from colorama import Fore, Style

cfg=utils.load_config('config/config_CAN.yaml')
utils.print_banner(f"Supported Regions for {cfg.get('country')}")
print(Fore.YELLOW + ">> RegionCode: name <<" + Style.RESET_ALL + "\n")
for keys in cfg.get('region_mapping'):
    print(f"{keys}: {cfg.get('region_mapping').get(keys).get('name')}")


# Set Region

In [ ]:
# The tool is designed to work for WB6 regions
region_code='AB'
region_name=cfg.get('region_mapping').get(region_code).get('name')
utils.print_banner(f"Selected Region: {region_name} ({region_code})")

### Load store (hdf5 file)

In [ ]:

store=f"./data/store/resources_{region_code}.h5" 
utils.print_banner(f"Store loaded for {region_name} ({region_code})")
res_data=DataHandler(store) # the DataHandler object could be initiated without the store definition as well.
res_data.show_tree(store)


### Load dataframes from Store

In [ ]:
# Loading dataframes
cells=res_data.from_store('cells')
boundary=res_data.from_store('boundary')
lines=res_data.from_store('lines')
ss=res_data.from_store('substations')
timeseries_clusters_solar=res_data.from_store('timeseries/clusters/solar')
timeseries_clusters_wind=res_data.from_store('timeseries/clusters/wind')
clusters_solar=res_data.from_store('clusters/solar')
clusters_wind=res_data.from_store('clusters/wind')

# Create Visuals

## Grid

In [ ]:
# for columns in lines.columns:
#     print(f"{columns}")

### lines

In [ ]:
# import matplotlib.pyplot as plt
# import matplotlib.patches as mpatches

# fig, ax = plt.subplots(figsize=(10, 8))
# boundary.plot(ax=ax, facecolor='grey', edgecolor='black', linewidth=1, alpha=0.1)

# if 'voltage' in lines.columns:
#     # Convert to numeric and drop NaNs
#     voltages = pd.to_numeric(lines['voltage'], errors='coerce').dropna().unique()
#     voltages.sort()
    
#     # Convert to kilovolts (kV)
#     voltages_kv = voltages / 1000

#     # Use a qualitative color map with enough distinct colors
#     cmap = plt.cm.get_cmap('Dark2', len(voltages_kv))  # tab20 has 20 distinct colors

#     color_map = {v_orig: cmap(i) for i, v_orig in enumerate(voltages)}

#     # Plot each voltage level with its unique color
#     for v_orig in voltages:
#         color = color_map[v_orig]
#         mask = pd.to_numeric(lines['voltage'], errors='coerce') == v_orig
#         lines[mask].plot(ax=ax, color=color, linewidth=1.5, alpha=0.8)

#     # Create legend patches
#     legend_patches = [
#         mpatches.Patch(color=color_map[v], label=f"{int(v / 1000)} kV") for v in voltages
#     ]
#     ax.legend(handles=legend_patches, title='Voltage', frameon=False)
#     ax.set_title(f'Transmission Lines by Voltage (kV) for {country_name}', fontsize=14)

# else:
#     lines.plot(ax=ax, color='blue', linewidth=1)
#     ax.set_title('Transmission Lines', fontsize=14)

# ax.set_axis_off()
# plt.tight_layout()

# plt.savefig(f"vis/network/transmission_lines_{country_code}.png", bbox_inches='tight', dpi=300)
# plt.show()

### Substations

In [ ]:
# <code>

## Resources

### Capacity Factor Map

* Individual Maps

In [ ]:
vis.get_data_in_map_plot(cells=cells, 
                resource_type='solar', 
                datafield='CF',
                title=f"Solar Resources for {region_name}",
                # ax=ax1, 
                show=False)

In [ ]:
# vis.get_data_in_map_plot(cells, 
#                 resource_type='wind', 
#                   datafield='CF',
#                 title=f"Wind Resources for {region_name}",
#                 # ax=ax1, 
#                 show=False)

* Combined Map

In [ ]:
import matplotlib.pyplot as plt

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(18, 8))

fig.suptitle(f"Resources for {region_name}", fontsize=18, fontweight='bold')

vis.get_data_in_map_plot(cells, 
                resource_type='solar',
                datafield='CF',
                ax=ax1, 
                show=False)
vis.get_data_in_map_plot(cells, 
                resource_type='wind',
                datafield='CF',
                ax=ax2, 
                show=False)

plt.tight_layout()

plt.show()


### Capacity

In [ ]:
vis.get_data_in_map_plot(cells, 
                resource_type='solar',
                datafield='capacity',
                # ax=ax1, 
                show=False)

In [ ]:
import matplotlib.pyplot as plt

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(18, 8))

fig.suptitle(f"Resources for {region_name}", fontsize=18, fontweight='bold')

vis.get_data_in_map_plot(cells, 
                resource_type='solar',
                datafield='capacity',
                ax=ax1, 
                show=False)
vis.get_data_in_map_plot(cells, 
                resource_type='wind',
                datafield='capacity',
                ax=ax2, 
                show=False)

plt.tight_layout()

plt.show()


### Score

In [ ]:
vis.get_data_in_map_plot(cells, 
                resource_type='solar',
                datafield='score',
                # ax=ax1, 
                show=False)

#### Capacity Plot (bins)

* Not required for now

In [ ]:
# import matplotlib.pyplot as plt
# import pandas as pd

# legend_x_ax_offset=1.1

# # Ensure 'Region' is in the columns for both boundary and cells
# # if boundary is not None and ('Region' not in boundary.columns or 'Country' not in boundary.columns):
# #     boundary = boundary.reset_index()

# # Assign a number to each region
# # boundary['Region_Number'] = range(1, len(boundary) + 1)

# # Define custom bins and labels for solar and wind capacity
# solar_bins = [0, 100, 200, 300, 500, float('inf')]  # Custom ranges
# solar_labels = ['<100','100-200', '200-300', '300-500','>500']  # Labels for legend

# # Define custom bins and labels for solar and wind capacity
# wind_bins = [0, 300, 500, 1000, 2000,3000, float('inf')]  # Custom ranges
# wind_labels = ['<300','300-500', '500-1000', '1000-2000','2000-3000', '>3000']  # Labels for legend

# # Categorize potential_capacity_solar and potential_capacity_wind into bins
# clusters_solar['solar_category'] = pd.cut(clusters_solar['potential_capacity'], bins=solar_bins, labels=solar_labels, include_lowest=True)
# clusters_wind['wind_category'] = pd.cut(clusters_wind['potential_capacity'], bins=wind_bins, labels=wind_labels, include_lowest=True)

# # Create figure and axes for side-by-side plotting
# fig, (ax1, ax2) = plt.subplots(figsize=(18, 8), ncols=2)
# fig.suptitle("Potential Sites for Targeted Capacity Investments", fontsize=16,weight='bold')
# # Set axis off for both subplots
# ax1.set_axis_off()
# ax2.set_axis_off()

# # Shadow effect offset
# shadow_offset = 0.01

# # Plot solar map on ax1
# # Add shadow effect for solar map
# boundary.geometry = boundary.geometry.translate(xoff=shadow_offset, yoff=-shadow_offset)
# boundary.plot(ax=ax1, color='None', edgecolor='grey', linewidth=1, alpha=0.7)  # Shadow layer
# boundary.geometry = boundary.geometry.translate(xoff=-shadow_offset, yoff=shadow_offset)

# # Plot solar cells
# clusters_solar.plot(column='solar_category', ax=ax1, cmap='Wistia', legend=True, 
#            legend_kwds={'title': "Solar Capacity (MW)", 'loc': 'upper right','fontsize':12,'bbox_to_anchor':(legend_x_ax_offset,1), 'frameon': False})

# # Plot actual boundary for solar map
# # boundary.plot(ax=ax1, facecolor='none', edgecolor='black', linewidth=0.2, alpha=0.7)
# """
# # Annotate region numbers for solar map
# for idx, row in boundary.iterrows():
#     centroid = row.geometry.centroid
#     ax1.annotate(f"{row['Region_Number']}", 
#                  xy=(centroid.x, centroid.y), 
#                  ha='center', va='center',
#                  fontsize=7, color='black',
#                  bbox=dict(facecolor='white', edgecolor='none', alpha=0.7, boxstyle='round,pad=0.2'))
# """
# # Plot wind map on ax2
# # Add shadow effect for wind map
# boundary.geometry = boundary.geometry.translate(xoff=shadow_offset, yoff=-shadow_offset)
# boundary.plot(ax=ax2, color='None', edgecolor='grey', linewidth=1, alpha=0.7)  # Shadow layer
# boundary.geometry = boundary.geometry.translate(xoff=-shadow_offset, yoff=shadow_offset)

# # Plot wind cells
# clusters_wind.plot(column='wind_category', ax=ax2, cmap='summer', legend=True, 
#            legend_kwds={'title': "Wind Capacity (MW)", 'fontsize':12,'bbox_to_anchor':(legend_x_ax_offset,1), 'frameon': False})

# # Plot actual boundary for wind map
# boundary.plot(ax=ax2, facecolor='none', edgecolor='black', linewidth=0.2, alpha=0.7)
# """
# # Annotate region numbers for wind map
# for idx, row in boundary.iterrows():
#     centroid = row.geometry.centroid
#     ax2.annotate(f"{row['Region_Number']}", 
#                  xy=(centroid.x, centroid.y), 
#                  ha='center', va='center',
#                  fontsize=8, color='black',
#                  bbox=dict(facecolor='white', edgecolor='none', alpha=0.7, boxstyle='round,pad=0.2'))
# """
# # Adjust layout for cleaner appearance
# fig.patch.set_alpha(0)  # Make figure background transparent
# plt.tight_layout()


# # Add annotation for solar capacity
# ax1.annotate(f"Targeted Capacity: \n{int(clusters_solar.potential_capacity.sum()/1e3)} GW",
#              xy=(1.2, 0.5), xycoords='axes fraction', ha='center', 
#              fontsize=14, color='black', fontweight='bold')

# # Add annotation for wind capacity
# ax2.annotate(f"Targeted Capacity: \n{int(clusters_wind.potential_capacity.sum()/1e3)} GW",
#              xy=(1.2, 0.5), xycoords='axes fraction', ha='center', 
#              fontsize=14, color='black', fontweight='bold')
# # Show the side-by-side plot

# plt.savefig('vis/solar_wind_capacity_map.png',dpi=300)
# # Add a directional compass (north arrow) to both subplots
# # Use a more standard north arrow style
# vis.add_compass_to_plot(ax1)
# vis.add_compass_to_plot(ax2)
# plt.show()

* Not required for now

In [ ]:
# import matplotlib.pyplot as plt
# import pandas as pd

# legend_x_ax_offset = 1.1
# shadow_offset = 0.01

# # Create figure and axes
# fig, (ax1, ax2) = plt.subplots(figsize=(18, 8), ncols=2)
# fig.suptitle("Potential Sites for Targeted Capacity Investments", fontsize=16, weight='bold')
# ax1.set_axis_off()
# ax2.set_axis_off()

# # --- Solar Map ---
# boundary.geometry = boundary.geometry.translate(xoff=shadow_offset, yoff=-shadow_offset)
# boundary.plot(ax=ax1, color='None', edgecolor='grey', linewidth=0.2, alpha=0.7)
# boundary.geometry = boundary.geometry.translate(xoff=-shadow_offset, yoff=shadow_offset)

# clusters_solar.plot(
#     column='potential_capacity',
#     ax=ax1,
#     cmap='Wistia',
#     legend=True,
#     legend_kwds={'label': "Solar Capacity (MW)", 'shrink': 0.7}  # valid kwargs for colorbar
# )


# # --- Wind Map ---
# boundary.geometry = boundary.geometry.translate(xoff=shadow_offset, yoff=-shadow_offset)
# boundary.plot(ax=ax2, color='None', edgecolor='grey', linewidth=0.2, alpha=0.7)
# boundary.geometry = boundary.geometry.translate(xoff=-shadow_offset, yoff=shadow_offset)

# clusters_wind.plot(
#     column='potential_capacity',
#     ax=ax2,
#     cmap='summer',
#     legend=True,
#     legend_kwds={'label': "Wind Capacity (MW)", 'shrink': 0.7}
# )


# boundary.plot(ax=ax2, facecolor='none', edgecolor='black', linewidth=0.2, alpha=0.7)

# # Add capacity annotations
# ax1.annotate(f"Targeted Capacity: \n{int(clusters_solar.potential_capacity.sum()/1e3)} GW",
#              xy=(1.2, 0.5), xycoords='axes fraction', ha='center',
#              fontsize=14, color='black', fontweight='bold')

# ax2.annotate(f"Targeted Capacity: \n{int(clusters_wind.potential_capacity.sum()/1e3)} GW",
#              xy=(1.2, 0.5), xycoords='axes fraction', ha='center',
#              fontsize=14, color='black', fontweight='bold')

# fig.patch.set_alpha(0)
# plt.tight_layout()
# plt.savefig('vis/solar_wind_capacity_map.png', dpi=300)

# # Add compass
# vis.add_compass_to_plot(ax1)
# vis.add_compass_to_plot(ax2)

# plt.show()


In [ ]:
# Example of correct argument dictionary (if you want to use kwargs)
scatter_plot_args = {
    "solar_clusters": clusters_solar,
    "wind_clusters": clusters_wind,
    "bubbles_GW": [0.5, 1, 2, 5, 10], # bubble sizes in GW
    "bubbles_scale": 0.025, #100 times smaller than the original scale 
    "lcoe_threshold": 150, # LCOE threshold in $/MWh
    "save_to_root": "vis"
}

# Call the function using the argument dictionary
vis.plot_resources_scatter_metric_combined(**scatter_plot_args)

## CF checks

In [ ]:
gwa_country_code=cfg.get('region_mapping').get(region_code).get('GWA_country_code')
utils.print_banner(f"GWA Country Code Selected: {gwa_country_code}")

### Wind

In [ ]:
# Get total bounds from boundary GeoDataFrame
minx, miny, maxx, maxy = boundary.total_bounds

# Create bounding_box_dict with correct keys for downstream use
bounding_box_dict = {
    "minx": float(minx),
    "miny": float(miny),
    "maxx": float(maxx),
    "maxy": float(maxy)
}


In [ ]:
import rioxarray as rxr

raster_path=f'./data/downloaded_data/GWA/{gwa_country_code}_capacity-factor_IEC3.tif'
gwa_raster_data = (
        rxr.open_rasterio(raster_path)
        .rio.clip_box(**bounding_box_dict)
        .rename('CF_IEC3')
        .drop_vars(['band', 'spatial_ref'])
        .isel(band=1 if '*Class*' in 'CF_IEC3' else 0)  # 'IEC_Class_ExLoads' data is in band 1
    )


In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(10, 6))
gwa_raster_data.plot(ax=ax, cmap='BuPu', add_colorbar=True)
boundary.plot(ax=ax, facecolor='none', edgecolor='white', linewidth=0.5)
ax.set_title('GWA CF-IEC3 Reference (High-res)', fontsize=11)
ax.axis('off')
plt.show()

In [ ]:
columns = ['CF_IEC2', 'CF_IEC3', 'wind_CF_mean']
cells[columns].describe().style.format(precision=2).set_caption("Summary Statistics for GWA CF data and calibrated Wind CF_mean")

In [ ]:
import xarray
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.gridspec import GridSpec
from IPython.display import display

def get_CF_wind_check_plot(cells: gpd.GeoDataFrame,
                           gwa_raster_data: xarray.DataArray,
                           boundary: gpd.GeoDataFrame,
                           region_name: str,
                           columns: list,
                           figure_height: int = 7,
                           compass_size: int = 8):
    """
    Plots GWA benchmark (left), CF_IEC3 (middle), and wind_CF_mean (right).
    """
    import vis  # assumes vis.add_compass_to_plot() exists
    
    assert len(columns) == 2, "Expected exactly two columns: CF_IEC3 and wind_CF_mean"
    col_mid, col_right = columns

    # Color scale
    vmin = cells[columns].min().min()
    vmax = cells[columns].max().max()

    # Layout: 1 row × 3 columns
    fig = plt.figure(figsize=(13, figure_height), constrained_layout=True)
    spec = GridSpec(nrows=1, ncols=3, width_ratios=[1, 1, 1], figure=fig)


    axes = []

    # LEFT: GWA benchmark
    ax_gwa = fig.add_subplot(spec[0, 0])
    gwa_raster_data.plot(ax=ax_gwa, cmap='BuPu', vmin=vmin, vmax=vmax, add_colorbar=False)
    boundary.plot(ax=ax_gwa, facecolor='none', edgecolor='white', linewidth=0.5)
    ax_gwa.set_title('GWA CF-IEC3 Reference (High-res)', fontsize=11)
    ax_gwa.axis('off')
    # vis.add_compass_to_plot(ax_gwa, size=compass_size, triangle_size=0.02)
    axes.append(ax_gwa)

    # MIDDLE: CF_IEC3
    ax_mid = fig.add_subplot(spec[0, 1])
    shadow_offset = 0.02
    cells_shadow = cells.copy()
    cells_shadow['geometry'] = cells_shadow['geometry'].translate(xoff=-shadow_offset, yoff=shadow_offset)
    cells_shadow.plot(column=col_mid, cmap='Greys', ax=ax_mid, edgecolor='white', alpha=1, linewidth=0.2, zorder=1)

    cells.plot(
        column=col_mid,
        ax=ax_mid,
        cmap='BuPu',
        vmin=vmin, vmax=vmax,
        linewidth=0.2,
        legend=False
    )
    ax_mid.set_title(col_mid.replace('_', ' '), fontsize=10)
    ax_mid.axis('off')
    axes.append(ax_mid)

    # RIGHT: wind_CF_mean
    ax_right = fig.add_subplot(spec[0, 2])
    col = col_right
    cells_shadow = cells.copy()
    cells_shadow['geometry'] = cells_shadow['geometry'].translate(xoff=-shadow_offset, yoff=shadow_offset)
    cells_shadow.plot(column=col, cmap='Greys', ax=ax_right, edgecolor='white', alpha=1, linewidth=0.2, zorder=1)

    cells.plot(
        column=col,
        ax=ax_right,
        cmap='BuPu',
        vmin=vmin, vmax=vmax,
        linewidth=0.2,
        legend=False
    )
    ax_right.set_title(col.replace('_', ' '), fontsize=10)
    ax_right.axis('off')
    axes.append(ax_right)

    # Unified colorbar
    norm = mpl.colors.Normalize(vmin=vmin, vmax=vmax)
    sm = mpl.cm.ScalarMappable(cmap='BuPu', norm=norm)
    cbar = fig.colorbar(sm, ax=axes, orientation='vertical', fraction=0.025, pad=0.02)
    cbar.set_label('Capacity Factor', fontsize=11)

    # Title and notes
    plt.suptitle(f"Wind Capacity Factor Comparison for {region_name}", fontsize=14, fontweight='bold', y=1.04)
    plt.figtext(
        0.01, 0.01,
        "* CF_IEC3 is rescaled from GWA to ERA5 resolution.\n"
        "* wind_CF_mean is computed using atlite (ERA5 adjusted with GWA).",
        ha='left', fontsize=9, color='gray'
    )

    plt.show()

    # Summary table
    display(cells[columns].describe().style.format(precision=2).set_caption("Summary Statistics for CF_IEC3 and calibrated Wind CF_mean"))


In [ ]:
get_CF_wind_check_plot(cells, 
                       gwa_raster_data,
                       boundary,
                       region_name,
                       ['CF_IEC3', 'wind_CF_mean'],
                       figure_height=5)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from textwrap import fill

# Set a clean and minimal style
sns.set_style("white")

# Initialize the figure
plt.figure(figsize=(8, 4))

# Create the boxplot
ax = sns.boxplot(
    data=cells[['CF_IEC2', 'CF_IEC3', 'wind_CF_mean']],
    palette="pastel",
    linewidth=1.5,
    width=0.6
)

# Set title and labels
ax.set_title('Capacity Factor Distribution Comparison', fontsize=13, weight='semibold', pad=12)
ax.set_ylabel('Capacity Factor', fontsize=11)
ax.set_xlabel('')

# Tweak tick formatting
ax.tick_params(axis='x', labelsize=10)
ax.tick_params(axis='y', labelsize=10)

# Remove all spines
for spine in ax.spines.values():
    spine.set_visible(False)

# Add horizontal grid lines
ax.yaxis.grid(True, linestyle='--', alpha=0.4)
ax.xaxis.grid(False)

plt.figtext(
    0.01, -0.08,
    "*'CF_IEC2, CF_IEC3: Average CF for ERA5 Cells, calculated from high-resolution yearly average CF from GWA for IEC Class 2 and 3 turbines.\n"
    "*wind_CF_mean: Average CF for ERA5 Cells, calculated from the ERA5 windspeed (rescaled with GWA) time series ",
    ha='left', fontsize=9, style='normal', fontweight='normal', color='gray'
)

plt.tight_layout()
plt.show()


* Solar 

### CLusters

In [ ]:
clusters_wind_f=clusters_wind[clusters_wind['potential_capacity']>0]
clusters_solar_f=clusters_solar[clusters_solar['potential_capacity']>0]

In [ ]:
print(f'Total sites {len(clusters_wind_f)}')
total_capacity=clusters_wind_f.potential_capacity.sum()
print(f'Total Capacity {int(total_capacity/1E3)} GW')
sites=5
top_sites_capacity=clusters_wind_f.head(sites).potential_capacity.sum()
print(f'Top {sites} sites ({round(sites/len(clusters_wind_f)*100)}% site) capacity {int(top_sites_capacity/1E3)} GW ({round(top_sites_capacity/total_capacity*100)}% of total capacity)')

In [ ]:
print(f'Total sites {len(clusters_solar_f)}')
total_capacity=clusters_solar_f.potential_capacity.sum()
print(f'Total Capacity {int(total_capacity/1E3)} GW')
sites=5
top_sites_capacity=clusters_solar_f.head(sites).potential_capacity.sum()
print(f'Top {sites} sites ({round(sites/len(clusters_solar_f)*100)}% site) capacity {int(top_sites_capacity/1E3)} GW ({round(top_sites_capacity/total_capacity*100)}% of total capacity)')

# Static Plots

In [ ]:
aeroway=gpd.read_file(f'./data/downloaded_data/OSM/{region_code}_aeroway.geojson')

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(12, 6))
shadow_offset=0.05
# Plot boundary with subtle shadow effect
boundary.geometry = boundary.geometry.translate(xoff=shadow_offset, yoff=-shadow_offset)
boundary.plot(ax=ax, facecolor='grey', edgecolor='lightgray', linewidth=0.1, alpha=0.1)
boundary.geometry = boundary.geometry.translate(xoff=-shadow_offset, yoff=shadow_offset)
boundary.plot(ax=ax, facecolor='none', edgecolor='k', linewidth=0.1, alpha=1)

# Plot aeroway data
aeroway.plot(
    column='aeroway',
    ax=ax,
    legend=True,
    legend_kwds={
        'title': "Aeroway Lands with Buffer",
        'loc': 'lower left',
        'bbox_to_anchor': (1, 0.01),
        'frameon': False
    },
    alpha=0.8,
    cmap='Paired'
)

# Clean up plot
ax.set_title(f"Aeroway Lands in {region_name}", fontsize=16, fontweight='bold', pad=15)
ax.axis('off')
plt.tight_layout()
plt.savefig(f'./vis/misc/Aeroway_{region_name}.png', dpi=300)
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Set Seaborn style for better aesthetics
sns.set_style("whitegrid")  

# Create figure
fig, ax = plt.subplots(figsize=(20, 4), facecolor="#f9f9f9")  # Light background

# Define custom colors
colors = ['#1f77b4', '#ff6347']

# Plot using Seaborn
sns.lineplot(data=timeseries_clusters_solar, x=timeseries_clusters_solar.index, y=timeseries_clusters_solar.iloc[:, 0], ax=ax, color=colors[0], linewidth=1.5, alpha=0.8, )
sns.lineplot(data=timeseries_clusters_solar, x=timeseries_clusters_solar.index, y=timeseries_clusters_solar.iloc[:, 1], ax=ax, color=colors[1], linewidth=1.5, alpha=0.8,)

# Enhance aesthetics
ax.set_facecolor("#ffffff")  # Pure white plot area
ax.grid(True, linestyle="--", linewidth=0.6, alpha=0.6, color="gray")

# Labels & title
ax.set_title("Solar Sites", fontsize=16, color="black", fontweight="bold", pad=15)
# ax.set_xlabel("Time", fontsize=14, color="black", labelpad=10)
ax.set_ylabel("CF", fontsize=14, color="black", labelpad=10)

# Customize ticks
ax.tick_params(axis='x', colors="black", labelsize=12)
ax.tick_params(axis='y', colors="black", labelsize=12)

# Add a legend
ax.legend(facecolor="#f0f0f0", edgecolor="black", fontsize=12, loc="upper left", frameon=True)

# Show plot
plt.show()


In [ ]:
# import matplotlib.pyplot as plt
# import seaborn as sns

# # Set Seaborn style for better aesthetics
# sns.set_style("whitegrid")  

# # Create figure
# fig, ax = plt.subplots(figsize=(20, 4.5), facecolor="#f9f9f9")  # Light background

# # Define custom colors
# colors = ['#1f77b4', '#ff6347']

# # Plot using Seaborn
# sns.lineplot(data=timeseries_clusters_wind, x=timeseries_clusters_wind.index, y='Capital_1', ax=ax, color=colors[0], linewidth=1.5, alpha=0.8, label="Capital 1")
# sns.lineplot(data=timeseries_clusters_wind, x=timeseries_clusters_wind.index, y='PeaceRiver_1', ax=ax, color=colors[1], linewidth=1.5, alpha=0.8, label="Peace River 1")

# # Enhance aesthetics
# ax.set_facecolor("#ffffff")  # Pure white plot area
# ax.grid(True, linestyle="--", linewidth=0.6, alpha=0.6, color="gray")

# # Labels & title
# ax.set_title("Wind Sites", fontsize=16, color="black", fontweight="bold", pad=15)
# # ax.set_xlabel("Time", fontsize=14, color="black", labelpad=10)
# ax.set_ylabel("CF", fontsize=14, color="black", labelpad=10)

# # Customize ticks
# ax.tick_params(axis='x', colors="black", labelsize=12)
# ax.tick_params(axis='y', colors="black", labelsize=12)

# # Add a legend
# ax.legend(facecolor="#f0f0f0", edgecolor="black", fontsize=12, loc="upper left", frameon=True)

# # Show plot
# plt.show()


# Static Data Visuals in Interactive Maps

In [ ]:
"""
import hvplot.pandas
import holoviews as hv
from holoviews import opts
from bokeh.layouts import gridplot
from bokeh.io import show

# Initialize Holoviews extension
hv.extension('bokeh')


# Define a dictionary to map columns to specific colormaps
cmap_mapping = {
    'lcoe_wind': 'cool',
    'potential_capacity_wind': 'Blues',
    'lcoe_solar': 'autumn',
    'CF_IEC2': 'RdYlGn',
    'wind_CF_mean': 'RdYlGn',
    'windspeed_ERA5': 'winter',
    'nearest_station_distance_km': 'Oranges',
    'potential_capacity_wind': 'Blues',
    'potential_capacity_solar': 'Oranges',
}

# Define a function to create individual plots
def create_plot(column_name, cmap):
    return cells.hvplot(
        color=column_name,
        cmap=cmap,
        geo=True,
        tiles='CartoDark',  # Default base map
        frame_width=300,  # Adjust the size of the plots
        frame_height=300,  # Adjust the size of the plots
        data_aspect=.5,
        alpha=0.8,
        line_color='None',
        line_width=0.1,
        hover_line_color='red'
    ).opts(title=column_name,
            show_grid=True,
            show_legend=True,
            tools=['hover', 'pan', 'wheel_zoom','reset','box_select'],
            legend_position='top_right'
        )

# Create a list of plots for each column
plots = [create_plot(col, cmap) for col, cmap in cmap_mapping.items()]

# Create a grid layout for the plots
grid = hv.Layout(plots).cols(3)  # Adjust the number of columns as needed

# Show the layout
hv.save(grid, '../docs/grid_plots.html')  # Save the grid layout as an HTML file



# Render the layout as a Bokeh object
bokeh_layout = hv.render(grid, backend='bokeh')

# Show the layout
show(bokeh_layout)
"""

# Timeseries Plots

In [ ]:
# import pandas as pd
# import hvplot.pandas
# import panel as pn
# import random

# # Initialize Panel with the dark theme
# pn.extension(theme='default')

# # Load your DataFrames
# df_solar = timeseries_clusters_solar  # Your solar DataFrame
# df_wind = timeseries_clusters_wind    # Your wind DataFrame

# # Create a list of the column names for the dropdowns
# solar_options = df_solar.columns.tolist()
# wind_options = df_wind.columns.tolist()

# # Function to generate a random vibrant color
# def get_random_vibrant_color():
#     return "#{:02x}{:02x}{:02x}".format(random.randint(150, 255), random.randint(150, 255), random.randint(150, 255))


# # Create a function to update the solar plot based on the selected time series
# def update_solar_plot(selected_series):
#     return df_solar[selected_series].hvplot.line(
#         title=f"Time Series - Solar: {selected_series}",
#         xlabel="DateTime",
#         ylabel="Value",
#         legend='top_left',
#         width=1000,  # Width of the plot
#         height=200,  # Height of the plot
#         tools=['hover'],  # Enable hover tool
#         line_color=get_random_vibrant_color()  # Random vibrant color for the solar plot
#     )

# # Create a function to update the wind plot based on the selected time series
# def update_wind_plot(selected_series):
#     return df_wind[selected_series].hvplot.line(
#         title=f"Time Series - Wind: {selected_series}",
#         xlabel="DateTime",
#         ylabel="Value",
#         legend='top_left',
#         width=1000,  # Width of the plot
#         height=200,  # Height of the plot
#         tools=['hover'],  # Enable hover tool
#         line_color=get_random_vibrant_color()  # Random vibrant color for the wind plot
#     )

# # Create dropdown widgets for selecting the time series
# solar_dropdown = pn.widgets.Select(name='Select Solar Time Series', options=solar_options)
# wind_dropdown = pn.widgets.Select(name='Select Wind Time Series', options=wind_options)

# # Create dynamic panels that update the plot based on the dropdown selections
# dynamic_solar_plot = pn.bind(update_solar_plot, selected_series=solar_dropdown)
# dynamic_wind_plot = pn.bind(update_wind_plot, selected_series=wind_dropdown)

# # Create a layout with the dropdowns and the plots in a two-row grid
# grid_layout = pn.Column(
#     solar_dropdown, dynamic_solar_plot,  # Solar plot in the first row
#     wind_dropdown, dynamic_wind_plot     # Wind plot in the second row
# )

# # Save the grid layout as an HTML file
# grid_layout.save('../docs/time_series_plots.html')

# # Display the panel in a notebook or in a web application
# grid_layout.show()

In [ ]:
# import pandas as pd
# import hvplot.pandas
# import panel as pn
# import random

# # Initialize Panel with the dark theme
# pn.extension(theme='default')

# # Load your DataFrames
# df_solar = timeseries_clusters_solar  # Your solar DataFrame
# df_wind = timeseries_clusters_wind    # Your wind DataFrame
# clusters_solar = clusters_solar       # Your clusters_solar DataFrame

# # Create a list of the column names for the dropdowns
# solar_options = df_solar.columns.tolist()
# wind_options = df_wind.columns.tolist()

# # Function to generate a random vibrant color
# def get_random_vibrant_color():
#     return "#{:02x}{:02x}{:02x}".format(random.randint(150, 255), random.randint(150, 255), random.randint(150, 255))

# # Create a function to update the solar plot based on the selected time series
# def update_solar_plot(selected_series):
#     return df_solar[selected_series].hvplot.line(
#         title=f"Time Series - Solar: {selected_series}",
#         xlabel="DateTime",
#         ylabel="Value",
#         legend='top_left',
#         width=1000,  # Width of the plot
#         height=300,  # Increased height for better visibility
#         tools=['hover'],  # Enable hover tool
#         line_color=get_random_vibrant_color(),  # Random vibrant color for the solar plot
#         line_width=2,  # Make line thicker for better visibility
#         fontsize={'title': 16, 'xlabel': 14, 'ylabel': 14, 'legend': 12},  # Adjust font sizes
#     )

# # Create a function to update the wind plot based on the selected time series
# def update_wind_plot(selected_series):
#     return df_wind[selected_series].hvplot.line(
#         title=f"Time Series - Wind: {selected_series}",
#         xlabel="DateTime",
#         ylabel="Value",
#         legend='top_left',
#         width=1000,  # Width of the plot
#         height=300,  # Increased height for better visibility
#         tools=['hover'],  # Enable hover tool
#         line_color=get_random_vibrant_color(),  # Random vibrant color for the wind plot
#         line_width=2,  # Make line thicker for better visibility
#         fontsize={'title': 16, 'xlabel': 14, 'ylabel': 14, 'legend': 12},  # Adjust font sizes
#     )

# # Create dropdown widgets for selecting the time series
# solar_dropdown = pn.widgets.Select(name='Select Solar Time Series', options=solar_options, width=300)
# wind_dropdown = pn.widgets.Select(name='Select Wind Time Series', options=wind_options, width=300)


# # Create dynamic panels that update the plot based on the dropdown selections
# dynamic_solar_plot = pn.bind(update_solar_plot, selected_series=solar_dropdown)
# dynamic_wind_plot = pn.bind(update_wind_plot, selected_series=wind_dropdown)

# # Function to get relevant row data from clusters_solar
# def get_cluster_info(selected_series):
#     # Use the selected series name to find the relevant row in clusters_solar
#     selected_row = clusters_solar.loc[selected_series]
#     return selected_row

# # Create a function to update the table based on the selected time series
# def update_cluster_table(selected_series):
#     cluster_info = get_cluster_info(selected_series)
    
#     # Drop the geometry column if it exists
#     if 'geometry' in cluster_info.index:
#         cluster_info = cluster_info.drop('geometry')
    
#     # Return the DataFrame widget without the geometry column
#     return pn.widgets.DataFrame(cluster_info.to_frame().T, width=800, height=200)


# # Create dynamic panels for the table
# dynamic_cluster_table = pn.bind(update_cluster_table, selected_series=solar_dropdown)

# # Adjust the overall layout for better alignment and spacing
# grid_layout = pn.Column(
#     pn.Row(solar_dropdown, dynamic_solar_plot, align="center"),  # Center align solar dropdown and plot
#     pn.Row(dynamic_cluster_table),  # Show the table below the solar plot
#     pn.Row(wind_dropdown, dynamic_wind_plot, align="center"),    # Center align wind dropdown and plot
#     sizing_mode='stretch_width',  # Make layout responsive to different screen sizes
#     width=1100,  # Set a consistent width for the layout
#     height=900   # Adjust height to give more space for plots
# )

# # Save the grid layout as an HTML file
# grid_layout.save('../docs/time_series_plots_with_table.html')

# # Display the panel in a notebook or in a web application
# grid_layout.show()
